In [ ]:
import torch
import torch.nn as nn

In [ ]:
!gdown 1kTEftRBxz_dpMCcl8-JSOoGvD1yQBdvO
!gdown 1tADa7tvzVq5Lj7tDsYaHZY_i2ff8I-7E

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Chosen device:', device)

In [ ]:
X_train = torch.load('COST_X_train.pt')
X_train.shape

In [ ]:
Y_train = torch.load('COST_y_train.pt')
Y_train.shape

## CNN Regressor

In [ ]:
model.state_dict()['linear_1.weight'].dtype

In [ ]:
X = X_train[:7]
Y = Y_train[:7]

# X = X.permute(0, 3, 1, 2)

print(X.shape, Y.shape)

In [ ]:
### Utils ###
def k_mean(tensor):
    return torch.mean(tensor, dim=2)

def k_sqrt(tensor):
    r = torch.sqrt(tensor)
    return r


def k_atan(tensor):
    t = torch.atan2(tensor[0], tensor[1])
    return t


def ifft(x):
    y = torch.complex(x[:, :, :, 0], x[:, :, :, 1])
    ifft = torch.fft.ifft(y)
    return torch.stack([torch.real(ifft), torch.imag(ifft)], dim=3)



class CNN_Regressor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout_rate = 0.25
        self.num_complex_channels = 6
        # self.nn_input = Input((64, 100, 2))

        self.linear_1 = nn.Linear(8* 6* 64* 100, 64)
        self.linear_2 = nn.Linear(64, 32)
        self.linear_3 = nn.Linear(32, 2)

        self.flatten = nn.Flatten()



        # self.Conv3d(in_channels = self.num_complex_channels, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), strides=(1, 5, 1))
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.3)
        self.Dropout = nn.Dropout(self.dropout_rate)

        self.layD1 = nn.Conv3d(in_channels = 1, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD2 = nn.Conv3d(in_channels = 1, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD2_a = nn.Conv3d(in_channels = 16, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD2_b = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD3 = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD3_a = nn.Conv3d(in_channels = 16, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD3_b = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD4 = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layD4_a = nn.Conv3d(in_channels = 16, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layV1 = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layV2 = nn.Conv3d(in_channels = 16, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layV3 = nn.Conv3d(in_channels = 24, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layV4 = nn.Conv3d(in_channels = 32, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')
        self.layV5 = nn.Conv3d(in_channels = 40, out_channels = 8, kernel_size = (1, 23, self.num_complex_channels), stride=1, padding = 'same')


    def Conv3D(self, x, in_channels, out_channels, kernel_size, stride):
        input_size = torch.tensor(x.shape[1:4])
        output_size = input_size
        print("input_size, output_size, kernel_size, strides", input_size, output_size, kernel_size, stride)
        # pad_size = (2*(output_size-1) - input_size - torch.tensor(kernel_size))*(1 / torch.tensor(stride))
        # pad_size = (stride*(output_size - 1.) - input_size  + kernel_size)/2.
        pad_size = (torch.tensor([1., 1, 1]) * (output_size - 1.) - input_size + torch.tensor(kernel_size)) / 2.

        # print("PAD", pad_size)
        padding = tuple(pad_size.int())
        padding = (0, 11, 2, 0, 11, 3)
        # conv3d = nn.Conv3d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, stride=stride, padding = (0, 21,  3))
        conv3d = nn.Conv3d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, stride=1, padding = 'same')
        return(conv3d)

    ## Returns the value after forward pass
    def forward(self, x):
      # complex to polar
      real = x[:, :, :, 0]
      imag = x[:, :, :, 1]

      real_squared = real**2
      imag_squared = imag**2

      real_imag_squared_sum = real_squared + imag_squared
      print(real_imag_squared_sum.shape)

      # amplitude
      r = k_sqrt(real_imag_squared_sum)
      r = r.view(-1, 64, 100, 1)
      # print("r", r.shape)

      # phase
      t = k_atan([imag, real])
      t = t.view(-1, 64, 100, 1)
      # print("t", t.shape)

      polar_input = torch.cat((r, t), dim = 3)


      time_input = ifft(x)

      # print("x, polar_input, time_input  ", x.shape, polar_input.shape, time_input.shape)

      total_input = torch.cat((x, polar_input, time_input), dim = 3)
      # print("total", total_input.shape)

      # reduce dimension of time axis
      lay_input = total_input.reshape((-1, 64, 100, self.num_complex_channels, 1))
      # lay_input = total_input.reshape((-1, 64, 100, self.num_complex_channels))
      lay_input = lay_input.permute(0, 4, 3, 1, 2)
      # print("lay_input", lay_input.shape )




      # layD2 = Conv3D(8, (1, 23, 1), strides=(1, 5, 1), padding="same", kernel_regularizer=regularizers.l2(0.01))(layD2)

      D1 = self.layD1(lay_input)
      D1 = self.LeakyReLU(D1)
      D1 = self.Dropout(D1)

      D2 = self.layD2(lay_input)
      D2 = self.LeakyReLU(D2)
      # print("SHAPES")

      D2 = torch.concatenate((D1, D2), dim = 1)

      D2 = self.layD2_a(D2)
      D2 = self.LeakyReLU(D1)



      D2 = self.layD2_b(D2)
      # print("D2", D2.shape)



      D3 = self.layD3(D2)
      D3 = self.LeakyReLU(D3)
      D3 = self.Dropout(D3)
      # print("D3", D3.shape)

      D3 = torch.concatenate((D2, D3), dim = 1)


      D3 = self.layD3_a(D3)
      D3 = self.LeakyReLU(D3)
      D3 = self.Dropout(D3)


      D3 = self.layD3_b(D3)



      D4 = self.layD4(D3)
      D4 = self.LeakyReLU(D4)
      D4 = self.Dropout(D4)


      D4 = torch.concatenate((D4, D3), dim = 1)


      D4 = self.layD4_a(D4)


      V1 = self.layV1(D4)
      V1 = self.LeakyReLU(V1)
      V1 = self.Dropout(V1)

      V1 = torch.concatenate((V1, D4), dim = 1)

      V2 = self.layV2(V1)
      V2 = self.LeakyReLU(V2)
      V2 = self.Dropout(V2)


      V2 = torch.concatenate((V1, V2), dim = 1)


      V3 = self.layV3(V2)
      V3 = self.LeakyReLU(V3)
      V3 = self.Dropout(V3)

      V3 = torch.concatenate((V3, V2), dim = 1)


      V4 = self.layV4(V3)
      V4 = self.LeakyReLU(V4)
      V4 = self.Dropout(V4)

      V4 = torch.concatenate((V4, V3), dim = 1)


      V5 = self.layV5(V4)
      V5 = self.LeakyReLU(V5)
      V5 = self.Dropout(V5)

      # print(V5.shape)

      # y = V5.reshape(-1, 8* 6* 64* 100)
      y = self.flatten(V5)

      y = self.linear_1(y)
      y = torch.relu(y)

      y = self.linear_2(y)
      y = torch.relu(y)

      y = self.linear_3(y)

      return(y)

model = CNN_Regressor()


print(model)


In [ ]:
X.shape

In [ ]:
7*2*64*100

In [ ]:
Y_pred = model(X)

print(Y_pred.shape)

In [ ]:
x = torch.ones((15, 64, 100, 2))
print(x.shape)

y = model(x)

y.shape

In [ ]:
model.to(device)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Assuming X and Y are your input and output tensors respectively

# Create a TensorDataset
dataset = TensorDataset(X, Y)

# Define batch size
batch_size = 30

# Create DataLoader objects for handling batches
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
history = []

In [ ]:
# Define your loss function
criterion = nn.MSELoss()

# Define your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    total_loss = 0
    total_loss_ = 0
    total_loss_fft = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        batch_x = batch_x.to(device)
        output = model(batch_x)


        loss = criterion(output, batch_y.to(torch.float32))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    history.append(total_loss)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)} ")


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history[0:])

In [ ]:
model_basic = model
model_basic

In [ ]:
model_main = model

In [ ]:
try:
    X_enc_main = X_enc
except:
    2

In [ ]:
samplegap = 10

In [ ]:
X_train = X_train.to('cpu')

In [ ]:
with torch.no_grad():
    X_enc = model_main.to('cpu').encoder(X_train[::1].permute(0, 3, 1, 2))
print(X_enc.shape)
X_enc = torch.flatten(X_enc, start_dim =1)
print(X_enc.shape)

In [ ]:
X_enc.device

In [ ]:
plt.scatter(Y_train[:, 0], Y_train[:, 1])
plt.scatter(Y_train[::samplegap, 0], Y_train[::samplegap, 1])

plt.show()

print(X_enc[::samplegap].shape)

In [ ]:
torch.save(model_main, 'model_main_fft.pth')

In [ ]:
torch.save(model_main.state_dict, 'model_main_fft_state_dict.pt')

In [ ]:
model_main

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=10, max_depth=20)
model.fit(X_enc[::samplegap], Y_train[::samplegap])

In [ ]:
Y_pred = model.predict(X_enc)

In [ ]:
plt.figure(figsize = (15, 15))
plt.scatter(Y_train[:, 0], Y_train[:, 1], label = "GT")


plt.scatter(Y_pred[:, 0], Y_pred[:, 1], label = "Pred")

plt.scatter(Y_train[::samplegap, 0], Y_train[::samplegap, 1], label = "Labeled data", marker = 'x', color = 'yellow')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (15, 15))
# plt.scatter(Y_train[:, 0], Y_train[:, 1], label = "GT")


# plt.scatter(Y_pred[:, 0], Y_pred[:, 1], label = "Pred")
plt.scatter(Y_train[:, 0], Y_train[:, 1], label = "Loss", c = np.sum(Y_pred - Y_train.numpy(), axis = 1)**2)
# plt.scatter(Y_train[::samplegap, 0], Y_train[::samplegap, 1], label = "Labeled data", marker = 'x', color = 'yellow')
plt.colorbar()

plt.legend()
plt.show()

In [ ]:
model = RandomForestRegressor(n_estimators=10, max_depth=20)
model.fit(X_enc, Y_train)

In [ ]:
Y_pred = model.predict(X_enc)

In [ ]:
plt.figure(figsize = (15, 15))
plt.scatter(Y_train[:, 0], Y_train[:, 1], label = "GT")


plt.scatter(Y_pred[:, 0], Y_pred[:, 1], label = "Pred")

plt.legend()
plt.show()